In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np
import timm
import pandas as pd
from sklearn import svm
from sklearn.metrics import accuracy_score

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

data_dir = "temp"
dataset = ImageFolder(data_dir, transform=data_transforms)

# Split dataset into training and test sets
k = 5  # number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

fold_train_accs = []
fold_test_accs = []
fold_train_cms = []
fold_test_cms = []
train_features = []
test_features = []
train_labels = []
test_labels = []

for fold, (train_index, test_index) in enumerate(kf.split(dataset)):
    print(f"Fold {fold + 1}:")
    
    train_dataset = torch.utils.data.Subset(dataset, train_index)
    test_dataset = torch.utils.data.Subset(dataset, test_index)
    
    train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    test_dataloader = DataLoader(test_dataset, batch_size=8, shuffle=False)
    
    model = timm.create_model('vit_base_patch16_224', pretrained=True)
    model.to(device)
    
    num_epochs = 2
    learning_rate = 0.0001
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    
    for epoch in range(num_epochs):
        for i, (images, labels) in enumerate(train_dataloader):
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        print(f"Epoch {epoch + 1}, Loss: {loss.item()}")
    
    train_predictions = []
    test_predictions = []
    fold_train_features = []
    fold_test_features = []
    train_labels_fold = []
    test_labels_fold = []
    
    model.eval()
    with torch.no_grad():
        for images, labels in train_dataloader:
            images = images.to(device)
            labels = labels.to(device)
            features = model.head(model.forward_features(images))
            fold_train_features.extend(features.cpu().numpy())
            train_labels_fold.extend(labels.cpu().numpy())

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            train_predictions.extend(predicted.cpu().numpy())

        for images, labels in test_dataloader:
            images = images.to(device)
            labels = labels.to(device)
            features = model.head(model.forward_features(images))
            fold_test_features.extend(features.cpu().numpy())
            test_labels_fold.extend(labels.cpu().numpy())

            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            test_predictions.extend(predicted.cpu().numpy())

    train_acc = accuracy_score(train_labels_fold, train_predictions)
    test_acc = accuracy_score(test_labels_fold, test_predictions)
    train_cm = confusion_matrix(train_labels_fold, train_predictions)
    test_cm = confusion_matrix(test_labels_fold, test_predictions)
    
    fold_train_accs.append(train_acc)
    fold_test_accs.append(test_acc)
    fold_test_cms.append(test_cm)
    
    train_features.append(fold_train_features)
    test_features.append(fold_test_features)
    train_labels.append(train_labels_fold)
    test_labels.append(test_labels_fold)
avg_train_acc = np.mean(fold_train_accs)
avg_test_acc = np.mean(fold_test_accs)

avg_train_cm = np.mean(fold_train_cms, axis=0)
avg_test_cm = np.mean(fold_test_cms, axis=0)

print("Average Train Accuracy:", avg_train_acc)
print("Average Test Accuracy:", avg_test_acc)
print("Average Train Confusion Matrix:")
print(avg_train_cm)
print("Average Test Confusion Matrix:")
print(avg_test_cm)


In [ ]:
train_features = np.concatenate(train_features, axis=0)
test_features = np.concatenate(test_features, axis=0)
train_labels = np.concatenate(train_labels, axis=0)
test_labels = np.concatenate(test_labels, axis=0)
train_features_reshaped = train_features.reshape(train_features.shape[0] * train_features.shape[1], -1)
test_features_reshaped = test_features.reshape(test_features.shape[0] * test_features.shape[1], -1)
train_features_df = pd.DataFrame(train_features_reshaped)
test_features_df = pd.DataFrame(test_features_reshaped)
train_labels_df = pd.DataFrame(train_labels)
test_labels_df = pd.DataFrame(test_labels)
train_features_df.to_csv("vit_training_features.csv", index=False)
test_features_df.to_csv("vit_testing_features.csv", index=False)
train_labels_df.to_csv("vit_training_labels.csv", index=False)
test_labels_df.to_csv("vit_testing_labels.csv", index=False)

In [ ]:
import pandas as pd
from sklearn import svm
from sklearn.metrics import accuracy_score
train_features_df = pd.read_csv("vit_train_features.csv")
train_labels_df = pd.read_csv("vit_train_labels.csv")
test_features_df = pd.read_csv("vit_test_features.csv")
test_labels_df = pd.read_csv("vit_test_labels.csv")
train_features = train_features_df.values
train_labels = train_labels_df['label'].values
test_features = test_features_df.values
test_labels = test_labels_df['label'].values
clf = svm.SVC()
clf.fit(train_features, train_labels)
predictions = clf.predict(test_features)
accuracy = accuracy_score(test_labels, predictions)
print("Accuracy:", accuracy)